### Exercise 2 - Numerical Optimization

Consider the following nonlinear optimization problem
<img src="imgs/nlp.png" alt="nlp" width="350"/>


Compute the gradients of the objective $f = \frac{1}{2}(x-1)^2 + \frac{1}{2}(10(y - x^2))^2 + \frac{1}{2}x^2$ and the constraints $g = x + (1-y)^2 = 0$ and their Hessian on paper.
Write down on paper the Karush-Kuhn-Tucker (KKT) conditions for the above problem. Are these
conditions necessary for optimality? Are they sufficient?

Define CasADi functions for $f$ and $g$ and use CasADi to generate the gradient and hessian functions of these two functions.

In [17]:
import numpy as np
from casadi import SX, Function, sin, jacobian, vertcat, hessian

x = SX.sym('x')
y = SX.sym('y')

f= 0.5 * (x-1)**2 + 0.5*(10*(y-x*x))**2 + 0.5*x**2
g= x + (1-y)**2

f_func = Function('f', [x, y], [0.5 * (x-1)**2 + 0.5*(10*(y-x*x))**2 + 0.5*x**2])
g_func = Function('g', [x, y], [x + (1-y)**2])

print(f)
print(g)

z = vertcat(x, y)

f_grad = jacobian(f, z)
g_grad = jacobian(g, z)

f_hes, f_grad = hessian(f, z)
g_hes, g_grad = hessian(g, z)



@1=0.5, (((@1*sq((x-1)))+(@1*sq((10*(y-sq(x))))))+(@1*sq(x)))
(x+sq((1-y)))


Solve the above constrained optimization problem using CasADi using the solver `IPOPT`.

In [32]:
from casadi import nlpsol

nlp = {'x':z, 'f':f, 'g':g}
S = nlpsol('s', 'ipopt', nlp)
print(S)

r = S(x0=[0,1], lbg=0, ubg=0)
print(r)

x_opt = r['x']
lam_g_opt = r['lam_g']
print('x_opt: ', x_opt)

s:(x0[2],p[],lbx[2],ubx[2],lbg,ubg,lam_x0[2],lam_g0)->(x[2],f,g,lam_x[2],lam_g,lam_p[]) IpoptInterface
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bo

Make sure that the KKT conditions are satisfied at the solution that you obtain.

In [33]:
nlp_grad = S.get_function('nlp_grad')
print(nlp_grad)

nlp_grad:(x[2],p[],lam_f,lam_g)->(f,g,grad_gamma_x[2],grad_gamma_p[]) SXFunction


In [34]:
nlp_grad(x_opt, [], 1, lam_g_opt)

(DM(1.29347), DM(7.72782e-12), DM([-6.65676e-10, 5.04505e-09]), DM([]))

In [35]:
g_func(x_opt[0], x_opt[1])

DM(7.72782e-12)